# Deep Net

Now that we've got all the data, in arrays, and normalized, it's time to train a neural net.

In [20]:
import os
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
from IPython.display import display, Markdown
from time import time
import tensorflow as tf
%matplotlib inline

## Load preprocessed dataset

In [13]:
with open('data/combined.pickle', 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train']
y_train = data_dict['y_train']
X_test = data_dict['X_test']
y_test = data_dict['y_test']
X_valid = data_dict['X_valid']
y_valid = data_dict['y_valid']

del data_dict # Free all the mallocs!

print("Train shape: %s, %s" % (X_train.shape, y_train.shape))
print("Test shape: %s, %s" % (X_test.shape, y_test.shape))
print("Valid shape: %s, %s" % (X_valid.shape, y_valid.shape))

Train shape: (15000, 64, 64), (15000, 128, 128)
Test shape: (2500, 64, 64), (2500, 128, 128)
Valid shape: (2500, 64, 64), (2500, 128, 128)


## Add depth to tensor

In [16]:
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, dataset.shape[1], dataset.shape[2], 1)).astype(np.float32)
    labels = labels.reshape((-1, labels.shape[1], labels.shape[2], 1)).astype(np.float32)
    return dataset, labels

X_train, y_train = reformat(X_train, y_train)
X_test, y_test = reformat(X_test, y_test)
X_valid, y_valid = reformat(X_valid, y_valid)

print("Train shape: %s, %s" % (X_train.shape, y_train.shape))
print("Test shape: %s, %s" % (X_test.shape, y_test.shape))
print("Valid shape: %s, %s" % (X_valid.shape, y_valid.shape))

Train shape: (15000, 64, 64, 1), (15000, 128, 128, 1)
Test shape: (2500, 64, 64, 1), (2500, 128, 128, 1)
Valid shape: (2500, 64, 64, 1), (2500, 128, 128, 1)


## Define a CNN

In [23]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
dropout_prob = 0.7
intial_stddev = 0.1

graph = tf.Graph()

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
                          strides=[1, 2, 2, 1], padding='SAME')

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=intial_stddev))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=intial_stddev))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=intial_stddev))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = max_pool_2x2(tf.nn.relu(conv)) + layer1_biases
        hidden = tf.nn.dropout(hidden, dropout_prob)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = max_pool_2x2(tf.nn.relu(conv + layer2_biases))
        hidden = tf.nn.dropout(hidden, dropout_prob)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Learning rate with exponential decay
    global_step = tf.Variable(0) 
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

NameError: name 'valid_dataset' is not defined

In [ ]:
keep_training = True

# Accuracy arrays to draw a chart
minibatch_acc = [];
val_acc = [];
test_acc = [];

def show_stats():
    clear_output()
    handle1, = plt.plot(minibatch_acc, label="train score")
    handle2, = plt.plot(val_acc, label="validation score")
    plt.legend([handle1, handle2], loc="best")
    plt.show()

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    step = 0;
    while keep_training:
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset : batch_data, 
            tf_train_labels : batch_labels,
            global_step: step
        }
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        step = step + 1
        
        if (step % 200 == 0):
            show_stats()
            
        if (step % 50 == 0):
            m_acc = accuracy(predictions, batch_labels)
            minibatch_acc.append(m_acc)
            v_acc = accuracy(valid_prediction.eval(), valid_labels)
            val_acc.append(v_acc)
            #t_acc = accuracy(test_prediction.eval(), test_labels)
            #test_acc.append(t_acc)

            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % m_acc)
            print("Validation accuracy: %.1f%%" % v_acc)

    show_stats()
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

## Run this to stop training

In [17]:
keep_training = False